In [22]:
import sys
import os
project_root = "/Users/linarojas/Desktop/Research/Papers/Combinatorial_Ternary/Hybrid-Experimental-Data-Driven-Workflow"
sys.path.append(sys.path.append(project_root))

In [55]:
import pickle
import pandas as pd
import numpy as np

import torch
import torch.nn as nn 
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from Models.VAE.VAE_Model import VAE

Data Collection
1. Electronic distribution of the elements of the periodic table
2. Simulations from AFLOW, components, stechiometry, thermal conductivity
3. Extract the data AFLOW and convert it into a Dataframe (Drop Thermal Conductivity)
4. Organize the dataframe to display the element and the composition
5. Unify Electronic distribution and AFLOW Data and flat

In [ ]:
# 1. Electronic distribution of the periodic table

elec_struc = os.path.join(
    project_root,
    "Data_Extraction",
    "Electronic_configuration.pkl"
)


with open(elec_struc,"rb") as f:
    elec_data = pickle.load(f)

In [42]:
#2.Simulations from AFLOW
dataf_aflow = os.path.join(
    project_root,
    "Data_Extraction",
    "Data_Extraction_AFLOW.pkl"
)
data_aflow = pd.read_pickle(dataf_aflow)

In [ ]:
#3. Extract the data AFLOW and convert it into a Dataframe (Drop Thermal Conductivity)
keys = data_aflow.keys()

simulation_data = []

# Unifying the Dataframes
for key, df in data_aflow.items():
    subset = df[['species', 'stoichiometry', 'agl_thermal_conductivity_300K']].copy()
    subset['source'] = key   # optional: label where it came from
    simulation_data.append(subset)

simulation_data = pd.concat(simulation_data, ignore_index=True)
simulation_data.rename(columns={'agl_thermal_conductivity_300K':'thermal_conductivity'}, inplace=True)

## Splitting the Species and stoichiometry data
simulation_data["species_list"] = simulation_data["species"].str.replace(" ","").str.split(",")
simulation_data["stoich_list"] = simulation_data["stoichiometry"].str.replace(" ","").str.split(",")


In [ ]:
# 4. Organize the dataframe to display the element and the composition
unique_elements = simulation_data['species_list'].explode().unique().tolist()
unique_elements.append('Thermal_conductivity') # Adding the thermal conductivity column

df_composition = pd.DataFrame(columns=unique_elements)

## Build rows
row = []

for species_list,stoich_list,thermal in zip (simulation_data['species_list'],
                                             simulation_data['stoich_list'],
                                             simulation_data['thermal_conductivity']):
    row_dict = {}
    
    # Fill composition for each specie
    for element, compo in zip(species_list,stoich_list):
        row_dict[element] = compo
        df_composition[element] = compo
    
    # Fill missing species with 0
    for elem in unique_elements:
        if elem not in row_dict and elem != 'Thermal_conductivity':
            row_dict[elem] = 0
    
    # Add thermal conductivity
    #row_dict['Thermal_conductivity'] = thermal
    row.append(row_dict)
    
df_composition = pd.DataFrame(row)

In [ ]:
# 5. Unify Electronic distribution and AFLOW Data and flat 
df_new = pd.DataFrame()

blocks = []
elec_col_names =[] 
n_rows = len(df_composition)

for col in df_composition.columns:
    if col in elec_data and elec_data[col] is not None:
        
        mat = np.asarray(elec_data[col])
        flat = mat.ravel()
        flat = flat / (np.linalg.norm(flat) + 1e-8)
        

        # Create column names for this element’s matrix
        flat_cols = [f"{col}_m_{i}" for i in range(len(flat))]

        # Build a block: same flat vector repeated for each row of df_composition
        flat_block = pd.DataFrame(
            [flat] * n_rows,
            columns=flat_cols,
            index=df_composition.index
        )

        # Append: first matrix columns, then the composition column itself
        blocks.append(df_composition[[col]])
        blocks.append(flat_block)
        
        elec_col_names.extend(flat_cols)   
    else:
        # Column is not an element with electron matrix -> copy as is
        blocks.append(df_composition[[col]])

# Final dataframe: everything horizontally concatenated, in order
df_new = pd.concat(blocks, axis=1)

#df_new.to_pickle("Input_Data_VAE.pkl")

VAEs for the composition and electronic structure
1. Define training and loss function
2. Traning
3. Analyze weights
4. Open the saved model

In [ ]:
#1. Define training and loss function

def vae_loss(x,x_hat,mu,logvar, beta=1.0):
    # Reconstruction (MSE for continuous features)
    recon = F.mse_loss(x_hat, x, reduction="mean")
    # KL divrgence
    kl = -0.5 *torch.mean(1+logvar-mu.pow(2)-logvar.exp())
    return recon + beta*kl, recon, kl

def train_vae(X_np, latent_dim=16, beta=1.0, lr=1e-3, batch_size=128, epochs=200, device=None):
    device =device or ("cuda" if torch.cuda.is_available() else "cpu")
    
    X = torch.tensor(X_np, dtype=torch.float32)
    loader = DataLoader(TensorDataset(X), batch_size=batch_size, shuffle=True)
    
    model = VAE(input_dim=X.shape[1], latent_dim=latent_dim).to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    
    model.train()
    for epoch in range (1, epochs+1):
        total,rtot,ktot = 0.0,0.0,0.0
        for (xb,) in loader:
            xb = xb.to(device)
            x_hat, mu, logvar, _ = model(xb)
            loss, recon,kl = vae_loss(xb,x_hat,mu,logvar,beta=beta)
            
            opt.zero_grad()
            loss.backward()
            opt.step()
            
            total += loss.item()* xb.size(0)
            rtot += recon.item() * xb.size(0)
            ktot += kl.item() * xb.size(0)
            
        if epoch % 20 == 0 or epoch == 1:
            n = len(loader.dataset)
            print(f"Epoch {epoch:4d} | loss {total/n:.6f} | recon {rtot/n:.6f} | KL {ktot/n:.6f}")
        
        return model
    

In [ ]:
# 2. Tranning

df_X = df_new
X_np = df_X.values.astype("float32")

model = train_vae(X_np)
torch.save(model.state_dict(), "vae_model_AFLOW.pth")

In [ ]:
# 3. Analyze weights
model_path = os.path.join(
    project_root,
    "Models",
    "VAE",
    "vae_model_AFLOW.pth"
)
state = torch.load(model_path, map_location="cpu")

for name, weight in state.items():
    print(name, weight.shape)
    break

enc.0.weight torch.Size([256, 2204])


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = VAE(
    input_dim = 2204,
    latent_dim = 16,
    hidden_dims = (256,128)
)

state = torch.load(model_path, map_location=torch.device("cpu"))
model.to(device)
model.eval()

VAE(
  (enc): Sequential(
    (0): Linear(in_features=2204, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
  )
  (mu): Linear(in_features=128, out_features=16, bias=True)
  (logvar): Linear(in_features=128, out_features=16, bias=True)
  (dec): Sequential(
    (0): Linear(in_features=16, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=2204, bias=True)
  )
)

Extract encoder/latent variables
1. Load the data
2. Extract the encoder and latent variable
3. Generate New data

In [ ]:
# 1. Load data

# --- Load data ---
input_data = os.path.join(
    project_root,
    "Models",
    "VAE",
    "Input_Data_VAE.pkl"
)

df_new = pd.read_pickle(input_data)

Latent shape: (1848, 16)


In [ ]:
#2. Extract the encoder and latent variable
# Convert to tensor
X_np = df_new.values.astype("float32")
X = torch.tensor(X_np, dtype=torch.float32)

# --- DataLoader ---
dataloader = DataLoader(
    TensorDataset(X),
    batch_size=128,
    shuffle=False   # important for latent extraction
)

# --- Extract latent space ---
latents = []

model.eval()
with torch.no_grad():
    for (x,) in dataloader:
        x = x.to(device)

        h = model.enc(x)
        mu = model.mu(h)

        latents.append(mu.cpu())

Z = torch.cat(latents, dim=0).numpy()

print("Latent shape:", Z.shape) 


In [36]:
#2. Data Generation

n_samples = 100
latent_dim = 16 

z_generated = torch.randn(n_samples, latent_dim)

# Decoder
with torch.no_grad():
    generated_x = model.dec(z_generated)